In [ ]:
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q transformers
!pip install -q bitsandbytes

# Base model

In [1]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from transformers import pipeline

C:\Users\201902452\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#################################################################
# Tokenizer
#################################################################

model_name='meta-llama/Meta-Llama-3-8B-Instruct'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "bfloat16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Device partition
device_map = "auto"

In [3]:
#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [6]:
#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    low_cpu_mem_usage = True,
    device_map = "auto"
)

Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.81s/it]


In [7]:
def inference(messages):
    input_ids = tokenizer.apply_chat_template( #Apply LLama3-Instruct chat template
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    
    torch.backends.cuda.enable_mem_efficient_sdp(False)
    torch.backends.cuda.enable_flash_sdp(False)
    
    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    
    response = outputs[0][input_ids.shape[-1]:]
    
    return tokenizer.decode(response, skip_special_tokens=True)

In [8]:
messages = [
    {"role": "system", "content": "You are a helpful assistant helping individuals with their medical queries"},
    {"role": "user", "content": "I have been having alot of catching ,pain and discomfort under my right rib. If I twist to either side especially my right it feels like my rib actually catches on something and at times I have to stop try to catch my breath and wait for it to subside. There are times if I am laughing too hard that it will do the same thing but normally its more so if I have twisted or moved a certain way"},
]

In [9]:
print(inference(messages))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
C:\Users\201902452\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


I'm so sorry to hear that you're experiencing discomfort and catching sensations under your right rib. It's understandable that you're concerned about this, and I'm here to help you explore possible causes and potential solutions.

Based on your description, it's possible that you might be experiencing some kind of ribcage or diaphragmatic issue. Here are a few potential causes that come to mind:

1. Ribcage inflammation or irritation: The ribcage can become inflamed or irritated due to various reasons such as overuse, poor posture, or underlying conditions like fibromyalgia or costochondritis. This can cause pain and discomfort, especially when you twist or move in a certain way.
2. Diaphragmatic irritation or dysfunction: The diaphragm is a muscle that separates the chest cavity from the abdominal cavity. If it becomes irritated or dysfunctional, it can cause discomfort, pain, and difficulty breathing, especially when you twist or move. This might be related to underlying conditions 

In [10]:
messages = [
    {"role": "system", "content": "You are a helpful assistant helping individuals with their medical queries"},
    {"role": "user", "content": " Hi, ive just had an embryo replacement 5 days ago. SInce then i have had pain down my right leg near my groin , a kind of cold tinkly discomfort. Today i started getting reasonable stomach pains as well Is this suggestive of anything? does it mean the embryo hasn t taken? I have my blood test to confirm pregnany on day ten to come with thanks"},
]
print(inference(messages))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Hi there! I'm happy to help you with your concerns.

Firstly, congratulations on taking the first step towards building your family!

Regarding your symptoms, it's not uncommon to experience some discomfort or pain after an embryo replacement procedure. However, the symptoms you're describing are quite specific, and I'd like to address each one separately:

1. Pain down your right leg near your groin: This could be related to the embryo transfer procedure itself. The transfer process can cause some discomfort, especially if the embryo is placed in the correct location. It's also possible that you might be experiencing some minor bleeding or spotting, which can cause cramping or discomfort in the lower abdomen or groin area. However, it's essential to note that this symptom is not typically a sign of embryo failure.
2. Cold, tinkly discomfort: This is a bit more unusual, but it's possible that it's related to the embryo implantation process. Implantation can cause some discomfort, which

In [11]:
messages = [
    {"role": "system", "content": "You are a helpful assistant helping individuals with their medical queries"},
    {"role": "user", "content": " hi,i have itching between my legs and testicles,im facing this from past 2 years.im using antifungal powder for timebeing but it is not completely went off.now in that affected area some kind sticky liquid coming out and im unable to work properly.skin became black and looking like dried tree outer layer"},
]
print(inference(messages))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


I'm so sorry to hear that you're experiencing itching and discomfort between your legs and testicles. It's understandable that you're concerned about the sticky liquid and the skin changes.

Based on your description, it's possible that you may be experiencing symptoms of a chronic skin condition, such as eczema, psoriasis, or dermatitis. The sticky liquid and blackened skin could be indicative of an underlying skin issue.

Given that you've been using antifungal powder for the past two years, it's possible that the issue may not be purely fungal in nature. I would recommend consulting a dermatologist or a primary care physician to get a proper diagnosis and treatment plan.

In the meantime, here are some suggestions that may help alleviate your symptoms:

1. Keep the affected area clean and dry: Gently wash the area with a mild soap and lukewarm water, and pat it dry with a clean towel.
2. Apply a topical cream or ointment: Your doctor may prescribe a topical cream or ointment to help

In [12]:
messages = [
    {"role": "system", "content": "You are a helpful assistant helping individuals with their medical queries"},
    {"role": "user", "content": " Could you give a brief summary of what Trichomoniasis is?"},
]
print(inference(messages))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Trichomoniasis is a common sexually transmitted infection (STI) caused by a tiny parasite called Trichomonas vaginalis. It's usually spread through sexual contact with an infected person.

Here's a brief summary:

* Trichomoniasis is a curable infection that affects the genital area, particularly the vagina in women and the urethra in men.
* Symptoms can include:
	+ Abnormal vaginal discharge (thick, yellowish or greenish)
	+ Strong, unpleasant odor
	+ Pain or itching in the genital area
	+ Redness, swelling, or irritation of the vulva or vagina
	+ Painful urination
* In men, symptoms may be mild or absent, but can include:
	+ Mild discomfort or itching in the urethra
	+ Frequent urination
* Trichomoniasis can increase the risk of getting or transmitting other STIs, such as HIV.
* If left untreated, trichomoniasis can lead to complications like pelvic inflammatory disease (PID) in women or prostatitis in men.

It's essential to note that trichomoniasis can be asymptomatic, meaning it m

# Fine-tuned model

In [4]:
from huggingface_hub import notebook_login
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

fine_tuned_model = "MrFat/llama3_8b_medic-v0.3"

model_ft = AutoModelForCausalLM.from_pretrained(
    fine_tuned_model,
    quantization_config=bnb_config,
    device_map="auto",
)

Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 7/7 [00:10<00:00,  1.44s/it]


In [5]:
tokenizer_ft = AutoTokenizer.from_pretrained(
    fine_tuned_model,
    trust_remote_code=True,
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,
)
tokenizer_ft.pad_token = tokenizer_ft.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
def inference(messages):
    input_ids = tokenizer_ft.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model_ft.device)

    terminators = [
        tokenizer_ft.eos_token_id,
        tokenizer_ft.convert_tokens_to_ids("<|eot_id|>")
    ]
    
    torch.backends.cuda.enable_mem_efficient_sdp(False)
    torch.backends.cuda.enable_flash_sdp(False)
    
    outputs = model_ft.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    
    response = outputs[0][input_ids.shape[-1]:]
    
    return tokenizer_ft.decode(response, skip_special_tokens=True)

In [14]:
messages = [
    {"role": "system", "content": "You are a helpful assistant helping individuals with their medical queries"},
    {"role": "user", "content": "I have been having alot of catching ,pain and discomfort under my right rib. If I twist to either side especially my right it feels like my rib actually catches on something and at times I have to stop try to catch my breath and wait for it to subside. There are times if I am laughing too hard that it will do the same thing but normally its more so if I have twisted or moved a certain way"},
]
print(inference(messages))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
C:\Users\201902452\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


 Hi. Thanks for your query. I have gone through your question. It seems that you are suffering from musculoskeletal problem. You should consult a doctor and get done X-ray chest, lateral view and CT thorax. Based on the results of these investigations, you may need to undergo a surgical procedure. Hope I have answered your query. If you have doubt or have additional questions, I will be happy to answer. Please feel free to contact me. Wish you good health.   Take care.   Regards.   Chat Doctor.   .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  

In [15]:
messages = [
    {"role": "system", "content": "You are a helpful assistant helping individuals with their medical queries"},
    {"role": "user", "content": " Hi, ive just had an embryo replacement 5 days ago. SInce then i have had pain down my right leg near my groin , a kind of cold tinkly discomfort. Today i started getting reasonable stomach pains as well Is this suggestive of anything? does it mean the embryo hasn t taken? I have my blood test to confirm pregnany on day ten to come with thanks"},
]
print(inference(messages))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


 Hello, Thanks for using Chat Doctor. I have gone through your query and I can understand your concern. There is no relation between your symptoms and embryo replacement. You may be having a pelvic inflammatory disease, and that may be causing the symptoms. I suggest you to take a course of antibiotics, such as amoxicillin and clavulanate potassium, and metronidazole, for 7 days. Take a painkiller, such as ibuprofen, and a muscle relaxant, such as diclofenac sodium, as needed. Hope this helps.   Let me know if I can assist you further.   Take care.   Wish you a very good health.   Regards,   Chat Doctor.   .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 

In [16]:
messages = [
    {"role": "system", "content": "You are a helpful assistant helping individuals with their medical queries"},
    {"role": "user", "content": " hi,i have itching between my legs and testicles,im facing this from past 2 years.im using antifungal powder for timebeing but it is not completely went off.now in that affected area some kind sticky liquid coming out and im unable to work properly.skin became black and looking like dried tree outer layer"},
]
print(inference(messages))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


 Hi. Thanks for using Chat Doctor. I have gone through your query and can understand your concern. Itching between legs and testicles is a common problem, especially in males. This is due to fungal infection like candidiasis or dermatitis. Fungal infection can be caused by the skin contact with fungus. In this condition, skin becomes black and dry, and itching is severe. Itching can be controlled by using antifungal cream containing clotrimazole or miconazole. You can use clotrimazole cream 1% BID for 2 weeks. If you are allergic to clotrimazole, then you can use miconazole cream 2% BID for 2 weeks. You can also use oral antifungal like fluconazole 150 mg OD for 2 weeks. Hope this helps. Take care.   Wish you good health.   Take care.   Regards,   Chat Doctor.   .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 

In [17]:
messages = [
    {"role": "system", "content": "You are a helpful assistant helping individuals with their medical queries"},
    {"role": "user", "content": " Could you give a brief summary of what Trichomoniasis is?"},
]
print(inference(messages))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Trichomoniasis is a common parasitic infection caused by Trichomonas vaginalis, a protozoan parasite. It is spread through sexual contact and can infect both men and women. The symptoms of trichomoniasis vary depending on the individual and the stage of infection. In women, the infection can cause: 1. Thick, yellowish-green vaginal discharge that may be frothy and have a strong odor. 2. Itching of the genital area. 3. Pain during urination. 4. Painful sexual intercourse. In men, trichomoniasis usually causes no symptoms. However, in rare cases, it can cause: 1. Thick, yellowish-green urethral discharge. 2. Pain during urination. Trichomoniasis can be treated with antibiotics. In women, the infection is usually treated with metronidazole, and in men, it is usually treated with tinidazole. In pregnant women, the infection is usually treated with metronidazole. Trichomoniasis is a common infection and is often asymptomatic. However, if left untreated, it can lead to complications such as 

In [18]:
messages = [
    {"role": "system", "content": "Asnwer truthfully"},
    {"role": "user", "content": " Is it safe to travel pregnant?"},
]
print(inference(messages))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Hi, Thanks for choosing Chat Doctor. It is safe to travel pregnant, but it is always advisable to avoid traveling during the first trimester. If you are already pregnant, then you should avoid traveling to areas with high altitude, high temperature, and high humidity. These areas can cause dehydration, which may lead to pregnancy complications. If you have to travel, then you should take necessary precautions such as taking multivitamin, wearing comfortable clothes, and taking regular breaks. Hope this helps. Take care.   Chat Doctor. . .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .

In [19]:
messages = [
    {"role": "system", "content": "Asnwer truthfully"},
    {"role": "user", "content": " Hi, I'm worried about cholesterol. What can i do to reduce it?"},
]
print(inference(messages))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


 Hi, I can understand your concern. Cholesterol is a type of fat that is present in the blood. It is usually measured in milligrams per deciliter (mg/dL). High cholesterol can increase the risk of heart disease. The best way to reduce cholesterol is to make lifestyle changes such as: - Diet: Eat a balanced diet that is low in saturated fats, cholesterol, and sodium. Include foods rich in fiber, vitamins, and minerals. - Exercise: Regular exercise can help to lower cholesterol. - Weight Loss: Losing weight if you are overweight or obese can help to lower cholesterol. - Quit Smoking: Smoking can increase the risk of heart disease. Quitting smoking can help to lower cholesterol. - Medications: Certain medications such as statins can help to lower cholesterol. Please consult your doctor for advice on how to lower your cholesterol. Hope this helps. Wish you a good health.   Take Care. Chat Doctor. .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .

In [11]:
messages = [
    {"role": "system", "content": "Asnwer truthfully"},
    {"role": "user", "content": " How is a quadratic equation solved?"},
]
print(inference(messages))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Hello there! I can help you with that. Quadratic equations can be solved by using the quadratic formula. This formula is given as follows:

x = (-b ± √(b2 - 4ac)) / 2a

Here, a is the coefficient of x2, b is the coefficient of x, and c is the constant term. This formula is used to find the roots of the quadratic equation. If you have any doubt, feel free to ask.   Thanks for using Chat Doctor. Wish you a very good health. Take Care.   Chat Doctor. .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 